In [1]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoModel, AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from datasets import load_dataset
from transformers import DataCollatorWithPadding
import random
from tqdm import tqdm

2022-08-09 18:56:38.883533: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
strongbuy = pd.read_csv("./data/strongbuy.csv")
sell = pd.read_csv("./data/sell.csv")
holddown = pd.read_csv("./data/holddown.csv")

In [3]:
all_sell = pd.concat([sell, holddown])
all_data = pd.concat([strongbuy, all_sell])
all_data.sample(frac=1).reset_index(drop=True)

,filename,article,length,label
0,strongbuy51.txt,6월에도 우리나라의 폴리실리콘 수출 가격 상승 국내 정유/석유화학 기업 중 신재생에...,274,1
1,strongbuy431.txt,"이미 건축/주택부문의 영업이익은 분기당 1,500억원 이상 기록이 일반화되었으며, ...",510,1
2,strongbuy408.txt,"1Q16 Preview: 매출 6,053억원, 영업이익 111억원(OPM 1.8%)...",436,1
3,strongbuy42.txt,"두번째로, 대형평형 위주의 공급이었음에도 분양에 성공했다는 것이다. 이번 분양은 전...",205,1
4,strongbuy416.txt,연말 까지 200개 점포가 중간관리제로 전환될 경우 약 58억원 수준의 인건비를 절...,424,1
...,...,...,...,...
2745,strongbuy476.txt," 투자의견 STRONG BUY, 목표주가 146,000원 유지 한국항공우주 투자의...",512,1
2746,strongbuy133.txt,디스플레이 업체별 양극화 더 뚜렷해질 것 글로벌 패널업체들의 10.5세대 TFT 투...,584,1
2747,holddown111.txt,"탑라인 성장 전략 긍정적이지만, 제작비 증가는 수익성 부담 회사는 올해 가이던스로 ...",603,0
2748,strongbuy443.txt,외형성장보다 수익성 우선 동사는 무분별한 저가수주를 회피하며 수익성 향상을 추구한다...,485,1


In [4]:
print(f"{len(strongbuy)/len(strongbuy)} : {(len(sell) + len(holddown)) / len(strongbuy):.4f}")

1.0 : 0.6799


In [5]:
train_data, test_data = train_test_split(all_data, test_size=0.2, random_state=42, stratify=all_data[["label"]])

test_data["labels"] = test_data["label"]
test_data = test_data.drop(labels=["filename", "length", "label"], axis=1)

In [6]:
fold_length = 5
X = train_data.drop(labels=["filename", "length", "label"], axis=1)
y = train_data["label"]

skf = StratifiedKFold(n_splits=fold_length, shuffle=True, random_state=42)
fold_dataframe = list()

for fold_number, (train, valid) in enumerate(skf.split(X, y), 1):
    X_train, X_valid = X.iloc[train], X.iloc[valid]
    y_train, y_valid = y.iloc[train], y.iloc[valid]
    
    fold_train = X_train.loc[:]
    fold_train["labels"] = y_train
    
    fold_valid = X_valid.loc[:]
    fold_valid["labels"] = y_valid
    
    fold_train.to_csv(f"./data/train_data_fold{fold_number}.csv", index=False)
    fold_valid.to_csv(f"./data/valid_data_fold{fold_number}.csv", index=False)

test_data.to_csv(f"./data/test_data.csv", index=False)

In [7]:
kb_albert_model_path = "./model/kb-albert-char-base-v2"
albert = AutoModel.from_pretrained(kb_albert_model_path)
tokenizer = AutoTokenizer.from_pretrained(kb_albert_model_path)

tokenizer.truncation_side = "left"

Some weights of the model checkpoint at ./model/kb-albert-char-base-v2 were not used when initializing AlbertModel: ['predictions.bias', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'sop_classifier.classifier.bias', 'sop_classifier.classifier.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
MAX_LEN = 512

def tokenized_fn(data):
    outputs = tokenizer(data["article"], padding="max_length", max_length=MAX_LEN, truncation=True)
    outputs["labels"] = data["labels"]
    return outputs

dataset_list = list()
for fold_number in range(1, fold_length+1):
    train_dataset = load_dataset("csv", data_files=f"./data/train_data_fold{fold_number}.csv")["train"]
    valid_dataset = load_dataset("csv", data_files=f"./data/valid_data_fold{fold_number}.csv")["train"]

    train_dataset = train_dataset.map(tokenized_fn, remove_columns=["article"])
    valid_dataset = valid_dataset.map(tokenized_fn, remove_columns=["article"])
    
    dataset_list.append([train_dataset, valid_dataset])

test_dataset = load_dataset("csv", data_files=f"./data/test_data.csv")["train"]
test_dataset = test_dataset.map(tokenized_fn, remove_columns=["article"])

Using custom data configuration default-d03887d24e8e1d37


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /home/piai/.cache/huggingface/datasets/csv/default-d03887d24e8e1d37/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-d9ec5cbbb5b92d44


...

Using custom data configuration default-8750c0716a3a9f42


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /home/piai/.cache/huggingface/datasets/csv/default-8750c0716a3a9f42/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/550 [00:00<?, ?ex/s]

In [9]:
batch_size = 16

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

dataloader_list = list()
for train_fold, valid_fold in dataset_list:
    train_dataloader = torch.utils.data.DataLoader(
        train_fold,
        sampler = torch.utils.data.RandomSampler(train_fold),
        batch_size = batch_size,
        collate_fn = data_collator,
    )

    validation_dataloader = torch.utils.data.DataLoader(
        valid_fold,
        sampler = torch.utils.data.SequentialSampler(valid_fold),
        batch_size = batch_size,
        collate_fn = data_collator,
    )
    
    dataloader_list.append([train_dataloader, validation_dataloader])

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    sampler = torch.utils.data.SequentialSampler(test_dataset),
    batch_size = batch_size,
    collate_fn = data_collator,
)

In [10]:
class ClassificationHead(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.dropout = torch.nn.Dropout(0.25)
        self.out_proj = torch.nn.Linear(768, 2)
    
    def forward(self, features):
        # 보통 분류기에선 start 토큰에 분류 결과를 담음
        x = features[:, 0, :]    # take <s> token (equiv. to [CLS])
        x = x.reshape(-1, x.size(-1))
        x = self.dropout(x)

        x = self.out_proj(x)
        return x

class AInalyst(torch.nn.Module):
    def __init__(self, pretrained_model):
        super(AInalyst, self).__init__()
        self.pretrained = pretrained_model
        self.classifier = ClassificationHead()
    
    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.pretrained(
            input_ids=input_ids,
            attention_mask=attention_mask,
            # labels=labels
        )
        self.labels = labels
        logits = self.classifier(outputs["last_hidden_state"])
        # prob = torch.nn.functional.softmax(logits, dim=-1)
        
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
            return logits, loss
        else:
            return logits

In [11]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AInalyst(pretrained_model=albert)
model.to(device)
model = torch.nn.DataParallel(model)
isParallel = True

In [13]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

In [14]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [15]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

epochs = 5
for epoch in range(epochs):
    print(f"============ Epoch {epoch+1}/{epochs} ============")
    print("Training...")
    
    for fold_number, (train_dataloader, validation_dataloader) in enumerate(dataloader_list, 1):
        print(f"===== Epoch {epoch+1}/{epochs} - Fold {fold_number}/{fold_length} =====")
        total_train_loss = 0
        model.train()

        for step, batch in enumerate(train_dataloader):
            batch_input_ids = batch["input_ids"].to(device)
            batch_attention_mask = batch["attention_mask"].to(device)
            batch_labels = batch["labels"].to(device)

            model.zero_grad()

            logits, loss = model(
                input_ids = batch_input_ids,
                attention_mask = batch_attention_mask,
                labels = batch_labels,
            )

            if isParallel:
                loss = loss.mean()

            total_train_loss += loss.item()
            loss.backward()
            optimizer.step()

            # if step % 1000 == 0 and not step == 0:
            #     print("step : {:>5,} of {:>5,} loss: {:.5f}".format(step, len(train_dataloader), loss.item()))

        avg_train_loss = total_train_loss / len(train_dataloader)
        print()
        print(" Average training loss: {0:.5f}".format(avg_train_loss))

        # Validation
        print()
        print("Running Validation...")

        model.eval()
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        for step, batch in enumerate(validation_dataloader):
            batch_input_ids = batch["input_ids"].to(device)
            batch_attention_mask = batch["attention_mask"].to(device)
            batch_labels = batch["labels"].to(device)

            with torch.no_grad():
                logits, loss = model(
                    input_ids = batch_input_ids,
                    attention_mask = batch_attention_mask,
                    labels = batch_labels,
                )

                if isParallel:
                    loss = loss.mean()

                total_eval_loss += loss.item()
                logits = logits.detach().cpu().numpy()
                label_ids = batch_labels.to("cpu").numpy()
                total_eval_accuracy += flat_accuracy(logits, label_ids)

        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        print()
        print(" Valid Accuracy: {0:.5f}".format(avg_val_accuracy))
    
    # Test
    print(f"===== Epoch {epoch+1}/{epochs} - Test =====")
    print()
    print("Running Test...")

    model.eval()
    total_test_accuracy = 0
    total_test_loss = 0
    nb_test_steps = 0
    
    for step, batch in enumerate(test_dataloader):
        batch_input_ids = batch["input_ids"].to(device)
        batch_attention_mask = batch["attention_mask"].to(device)
        batch_labels = batch["labels"].to(device)

        with torch.no_grad():
            logits, loss = model(
                input_ids = batch_input_ids,
                attention_mask = batch_attention_mask,
                labels = batch_labels,
            )

            if isParallel:
                loss = loss.mean()

            total_test_loss += loss.item()
            logits = logits.detach().cpu().numpy()
            label_ids = batch_labels.to("cpu").numpy()
            total_test_accuracy += flat_accuracy(logits, label_ids)

    avg_test_accuracy = total_test_accuracy / len(test_dataloader)
    print()
    print(" Test Accuracy: {0:.5f}".format(avg_test_accuracy))
    print()

============ Epoch 1/5 ============
Training...
===== Epoch 1/5 - Fold 1/5 =====


/home/piai/anaconda3/envs/iml/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '



 Average training loss: 0.60353

Running Validation...

 Valid Accuracy: 0.80804
===== Epoch 1/5 - Fold 2/5 =====

 Average training loss: 0.32967

Running Validation...

 Valid Accuracy: 0.91295
===== Epoch 1/5 - Fold 3/5 =====

 Average training loss: 0.17486

Running Validation...

 Valid Accuracy: 0.96652
===== Epoch 1/5 - Fold 4/5 =====

 Average training loss: 0.08151

Running Validation...

 Valid Accuracy: 0.98438
===== Epoch 1/5 - Fold 5/5 =====

 Average training loss: 0.04464

Running Validation...

 Valid Accuracy: 0.99777
===== Epoch 1/5 - Test =====

Running Test...

 Test Accuracy: 0.92202

============ Epoch 2/5 ============
Training...
===== Epoch 2/5 - Fold 1/5 =====

 Average training loss: 0.01512

Running Validation...

 Valid Accuracy: 1.00000
===== Epoch 2/5 - Fold 2/5 =====

 Average training loss: 0.00634

Running Validation...

 Valid Accuracy: 1.00000
===== Epoch 2/5 - Fold 3/5 =====

 Average training loss: 0.00425

Running Validation...

 Valid Accuracy: 1

## Model Save

In [16]:
torch.save(model.state_dict(), "./model/kbalbert_epoch5_fold5.pt")

## Inference

In [17]:
load_model = AInalyst(pretrained_model=albert)
load_model.to(device)
load_model = torch.nn.DataParallel(load_model)
load_model.load_state_dict(torch.load("./model/kbalbert_epoch5_fold5.pt"))

<All keys matched successfully>

In [18]:
def inference_tokenized_fn(data):
    outputs = tokenizer(data["article"], padding="max_length", max_length=MAX_LEN, truncation=True)
    return outputs

inference_dataset = load_dataset("csv", data_files=f"./data/inference_data.csv")["train"]
inference_dataset = inference_dataset.map(inference_tokenized_fn,
                                          remove_columns=["Unnamed: 0", "company", "title", "opinion", "firm", "date", "article"])

Using custom data configuration default-68bfb0a5382dece9
Reusing dataset csv (/home/piai/.cache/huggingface/datasets/csv/default-68bfb0a5382dece9/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/piai/.cache/huggingface/datasets/csv/default-68bfb0a5382dece9/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-0c67678741f0cfd7.arrow


In [19]:
inference_dataloader = torch.utils.data.DataLoader(
    inference_dataset,
    sampler = torch.utils.data.SequentialSampler(inference_dataset),
    batch_size = 1,
    collate_fn = data_collator,
)

In [20]:
load_model.eval()

probabilities = list()
predictions = list()

for step, batch in enumerate(tqdm(inference_dataloader, desc="inference", mininterval=0.1)):
    batch_input_ids = batch["input_ids"].to(device)
    batch_attention_mask = batch["attention_mask"].to(device)

    with torch.no_grad():
        logits = load_model(
            input_ids = batch_input_ids,
            attention_mask = batch_attention_mask,
        )
        
        prob = torch.nn.functional.softmax(logits, dim=-1)
        predict = torch.argmax(prob, axis=1)
        
        prob = np.trunc(np.max(prob.detach().cpu().numpy(), axis=1) * 100)
        predict = predict.detach().cpu().numpy()
        
        probabilities.append(prob[0])
        predictions.append(predict[0])

inference: 100%|██████████████████████████| 50083/50083 [19:29<00:00, 42.81it/s]


In [21]:
inference_dataframe = pd.read_csv("./data/inference_data.csv")

convert_predictions = list(map(lambda x: "매수" if x == 1 else "매도", predictions))
inference_dataframe = inference_dataframe.drop(labels="Unnamed: 0", axis=1)
inference_dataframe["predictions"] = convert_predictions
inference_dataframe["pred_rate"] = probabilities
inference_dataframe.to_csv(f"./data/convert_inference_data.csv", index=False)

In [22]:
inference_dataframe["predictions"].value_counts()

매수    32975
매도    17108
Name: predictions, dtype: int64